# 0 Packete Laden

In [1]:
import pandas as pd

# Datensatz laden



In [2]:
df = pd.read_csv("/content/crossbooking_challenge.csv")

In [3]:
df.head(20)

,turnover_id,payee_payer_name,holder_name
0,b892f151-e54d-4cdc-ad3b-bb49c6f10197,Stefan und Ulrike Steffens,Stefan und Manuela Steffens
1,85921cb1-9d3c-4ee0-aefa-5b758eef8701,Drk Krejsverband Claire Siege V,Maren Hecker
2,63cb68a0-8a04-11ec-1d14-82bb5829b41b,Emslaendische Volksbank Eg,Hinrich Schleich
3,fe383450-bc4e-11ec-c954-6c39d3c8dc12,Mig Gmbh Und Co Fonds6 Kg,Käthe Finke
4,9a372a30-e9e2-11ec-e558-8689140089a8,Bundesagentur Rosel Arbeit Service Lehmann,Käthe Finke
5,a09add40-5239-11ec-7540-40403ed94418,Stilla Regina Rust,Stilla Regina Rust
6,d9f5d970-a8a9-11ec-9461-3fe2980e62fd,Ga Nr00002139 Blz45850005 2,Frithjof Herrmann
7,e79d2040-7e3a-11ec-005c-25b05d1c6034,Mastercard Abrechnung,Korbinian Hellwig
8,e491ab10-87a8-11ec-4643-1378609546af,Wuertt Versicherg Ag,Agnieszka Kurt Hartung
9,6cecb6c0-6dba-11ec-9b29-f42aa7e6703f,Aktion Adriane Arnfried 2022,Vincenzo Gröttner


## 1.Schritt: wir identifizieren alle exakt identischen Paare und speichern diese in einem Flag

In [4]:
df["is_identical"] = (df["payee_payer_name"] == df["holder_name"]).astype("int")

## 2.Schritt: wir identifizieren alle Paare bei denen ein Element NaN ist. Für diese Paare können wir kein Matching betreiben

In [5]:
df["at_leat_one_NaN"] = (df["payee_payer_name"].isna() | df["payee_payer_name"].isna()).astype("int")

## 3. Schritt: um ähnliche Namen zu identifizeren habe ich einen ngram-Overlap Ansatz gewählt. Konkret wird ein String dabei in Substrings von Länge n unterteilt. Damit kann man dann die Ähnlichkeit von 2 Strings vergleichen indem man die Schnittmenge in Relation zur Gesamtlänge setzt

In [6]:
n=3

#Initialisieren der Spalte mit 0
df["ngram3_overlap"] = 0

#Loop über die Zeilen des Data Frames:
for index, row in df.iterrows():

  #Payee & Holder werden von allen Sonderzeichen (.replace('\W', '')) und Leerzeichen (.replace(" ", "")) befreit
  payee = str(row['payee_payer_name']).replace('\W', '').replace(" ", "")
  holder = str(row['holder_name']).replace('\W', '').replace(" ", "")

  #Bildungs des 3grams für Payee und holder
  payee_3gram = [payee[i:i+n] for i in range(len(payee)-n+1)]
  holder_3gram = [holder[i:i+n] for i in range(len(holder)-n+1)]

  #Wir bestimmen die Overlaps zwischen den beiden Sets von 3grams und speichern diese als Liste 
  overlap = list(set(payee_3gram).intersection(set(holder_3gram)))


  #Wir berechnen den Overlap als Anteil der Overlap Menge an der 3Gram Menge des kürzeren der beiden Strings --> diesen Wert setzen wir dann in die entsprechnde Zeile & Spalte im df ein
  df.iloc[index, df.columns.get_loc("ngram3_overlap")] = len(overlap)/min([len(payee_3gram), len(holder_3gram)])


Ich habe mir nun angesehen wie gut dieser Ansatz funktioniert

In [7]:
#Dazu filtere ich auf solche Paare die nicht sowieso schon identisch sind und sortiere sie absteigend nach ngram3_overlap
df[df.is_identical==0].sort_values("ngram3_overlap", ascending=False).head(40)

,turnover_id,payee_payer_name,holder_name,is_identical,at_leat_one_NaN,ngram3_overlap
141,b892f151-e54d-4cdc-ad3b-bb49c6f10199,Herr Dr. Markus Reichhardt,Markus Reichhardt,0,0,1.000000
157,b892f151-e54d-4cdc-ad3b-bb49c6f10194,Stefan und Ulrike Steffens,Ulrike Steffens,0,0,1.000000
134,b892f151-e54d-4cdc-ad3b-bb49c6f10195,Stefan und Ulrike Steffens,UlrikeSteffens,0,0,1.000000
119,b892f151-e54d-4cdc-ad3b-bb49c6f10191,Hans Wurst,HansWurst,0,0,1.000000
80,b892f151-e54d-4cdc-ad3b-bb49c6f30394,Marcus Hofpfister,Marcus Hans-Peter Hofpfister,0,0,0.928571
71,b892f151-e54d-4cdc-ad3b-bb49c6f10184,Nathan Christin Zomniak,"Nathan, Christin Zomniak",0,0,0.894737
253,471c7770-5ecc-11ec-d7df-20e1df5570bc,Alfred Konstantin Ruppersberger,Alfred Ruppersberger,0,0,0.882353
149,b892f151-e54d-4cdc-ad3b-bb49c6f10294,Nele Wollershausen,"Nele Frida Wollershausen, Marcel Kunde",0,0,0.866667
274,108075e0-83bb-11ec-b50f-bf5f413300dd,Agnieszka Kurt Hartung,Agnieszka Hartung,0,0,0.857143
197,b892f151-e54d-4cdc-ad3b-bb49c6f10198,Stefan und Ulrike Steffens,"Steffens,Ulrike",0,0,0.769231


Der Ansatz scheint gut zu funktionieren um identische Konten zu identifizieren. 

*   Der Ansatz scheint gut zu funktionieren um identische Konten zu identifizieren. Alle Konten mit sehr hohen ngram3_overlap Werten scheinen Eigenüberweisungen zu sein
* Eine Ausnahme bildet die Überweisung zwischen "Stefan und Ulrike Steffens" und "Stefan und Ulrike Adenauer". Diese Fehlklassifikation sehe ich jedoch nicht als systematisches Problem, da ich es in der Realität für sehr unwahrscheinlich halte, dass Paare die exakt gleichen Vornamen und nur unterschiedliche Nachnamen haben
* Auch das Paar "Mann, Philip Tomas" und "TomasMann" ist recht ähnlich aber könnte eine Fehlklassifikation sein. Fakt ist aber, dass die beiden sehr ähnliche Namen haben.
*   Ab ngram3_overlap< 0.533333 scheint die Logik keine Eigenüberweisungen mehr zu identifizieren. Z.B. sind Herr Dr. Markus Reichhardt und Philipp Reichhardt	wohl nicht dieselben Personen --> wir setzen daher einen Cutoff Wert bei 0.55

Zur Sicherheit schauen wir uns nochmal Paare an, die eher kleine  ngram3_overlap Werte haben:

In [8]:
df[(df.is_identical==0) & (df.ngram3_overlap<0.55)].sort_values("ngram3_overlap", ascending=False).head(40)

,turnover_id,payee_payer_name,holder_name,is_identical,at_leat_one_NaN,ngram3_overlap
172,b892f151-e54d-4cdc-ad3b-bb49c6f10114,Herr Dr. Markus Reichhardt,Philipp Reichhardt,0,0,0.533333
0,b892f151-e54d-4cdc-ad3b-bb49c6f10197,Stefan und Ulrike Steffens,Stefan und Manuela Steffens,0,0,0.523810
196,b892f151-e54d-4cdc-ad3b-bb49c6f10134,"Leopold, Müller",Leopold Schmitz,0,0,0.416667
219,b892f151-e54d-4cdc-ad3b-bb49c6f10154,"Sabine, Macron",emmanuel macron,0,0,0.272727
88,9d746600-5ba7-11ec-7624-1eba8bebcfb6,Wuerttembergische Versicherung Ag,Alfred Ruppersberger,0,0,0.176471
139,4ebf9890-4739-11ec-06f1-4004c9ff3b98,Deutsche Klassenlotterie Berlin Rechtsf Anstal...,Toni Und Pirmin Pechel,0,0,0.117647
237,fb6b2ff0-40ef-11ec-f220-0ab150314f87,Deutsche Klassenlotterie Berlin Rechtsf Anstal...,Toni Und Pirmin Pechel,0,0,0.117647
100,4d151829-d295-4766-a337-6527a36fb398,Rewe Markt Gmbh,Maren Hecker,0,0,0.111111
103,823793e0-2bb8-11ec-f3b1-cce3d2656122,Deutsche Klassenlotterie Berlin Rechtsf Anstal...,Konstanze Und Kazim Holt,0,0,0.105263
267,84e168e1-0b31-4d29-a82c-9977120a2aa7,Techniker Krankenkasse,Reingard Henk,0,0,0.100000


--> sieht gut aus

# 4. Schritt Implementierung der Logik in einer Spalte

In [9]:
df["Eigenueberweisung"] = (( (df["is_identical"]==1) | (df["ngram3_overlap"]>=0.55) ) & (df["at_leat_one_NaN"]==0)).astype("int")

Checks ob alles geklappt hat:

In [10]:
df[df.Eigenueberweisung==1].tail(20)

,turnover_id,payee_payer_name,holder_name,is_identical,at_leat_one_NaN,ngram3_overlap,Eigenueberweisung
179,bd2a86a0-7d71-11ec-a3d5-18b6ba6fb852,Swetlana Weller,Swetlana Weller,1,0,1.000000,1
184,ef32acd0-66a7-11ec-400a-0fdfd96b0d33,Erik Ronny Geisler,Erik Ronny Geisler,1,0,1.000000,1
195,d284a600-3b6f-11ec-d42c-ce92dc57d179,Stilla Regina Rust,Stilla Regina Rust,1,0,1.000000,1
197,b892f151-e54d-4cdc-ad3b-bb49c6f10198,Stefan und Ulrike Steffens,"Steffens,Ulrike",0,0,0.769231,1
203,a0744270-5239-11ec-fa80-95d54f2ce503,Erik Ronny Geisler,Erik Ronny Geisler,1,0,1.000000,1
206,22da43a0-4b27-11ec-ed79-928a00fa9362,Stilla Regina Rust,Stilla Regina Rust,1,0,1.000000,1
209,28dc4670-384b-11ec-ff64-a2abc8dbe420,Amir Karz,Amir Karz,1,0,1.000000,1
210,d710e040-2d4a-11ec-9765-a24c48fa2a88,Eugenia Putz,Eugenia Putz,1,0,1.000000,1
213,1b570690-62ba-11ec-88fc-2109efba8b7e,Stilla Regina Rust,Stilla Regina Rust,1,0,1.000000,1
222,19980000-6771-11ec-c713-0c798a75e8d2,Swetlana Weller,Swetlana Weller,1,0,1.000000,1


In [11]:
df[df.Eigenueberweisung==0].head(20)

,turnover_id,payee_payer_name,holder_name,is_identical,at_leat_one_NaN,ngram3_overlap,Eigenueberweisung
0,b892f151-e54d-4cdc-ad3b-bb49c6f10197,Stefan und Ulrike Steffens,Stefan und Manuela Steffens,0,0,0.523810,0
1,85921cb1-9d3c-4ee0-aefa-5b758eef8701,Drk Krejsverband Claire Siege V,Maren Hecker,0,0,0.000000,0
2,63cb68a0-8a04-11ec-1d14-82bb5829b41b,Emslaendische Volksbank Eg,Hinrich Schleich,0,0,0.000000,0
3,fe383450-bc4e-11ec-c954-6c39d3c8dc12,Mig Gmbh Und Co Fonds6 Kg,Käthe Finke,0,0,0.000000,0
4,9a372a30-e9e2-11ec-e558-8689140089a8,Bundesagentur Rosel Arbeit Service Lehmann,Käthe Finke,0,0,0.000000,0
6,d9f5d970-a8a9-11ec-9461-3fe2980e62fd,Ga Nr00002139 Blz45850005 2,Frithjof Herrmann,0,0,0.000000,0
7,e79d2040-7e3a-11ec-005c-25b05d1c6034,Mastercard Abrechnung,Korbinian Hellwig,0,0,0.000000,0
8,e491ab10-87a8-11ec-4643-1378609546af,Wuertt Versicherg Ag,Agnieszka Kurt Hartung,0,0,0.000000,0
9,6cecb6c0-6dba-11ec-9b29-f42aa7e6703f,Aktion Adriane Arnfried 2022,Vincenzo Gröttner,0,0,0.000000,0
10,3c73fba0-7fcd-11ec-dd24-79982ba5fb42,Dm Drogeriemarkt Sagt Danke,Anna-Maria Scheuermann,0,0,0.000000,0


In [12]:
df.groupby("Eigenueberweisung")["turnover_id"].count()

Eigenueberweisung
0    209
1     73
Name: turnover_id, dtype: int64

--> der Anteil an Eigenüberweisungen scheint plausibel

# 5. Für bessere Wiederverwendbarkeit und schlankeren Code eine Funktion bauen:

In [13]:
def Eigenueberweisung(df, receiver="payee_payer_name", payer = "holder_name"):
  #Identical identifizeren
  df["is_identical"] = (df["payee_payer_name"] == df["holder_name"]).astype("int")
  
  #NaNs identifizieren
  df["at_leat_one_NaN"] = (df["payee_payer_name"].isna() | df["payee_payer_name"].isna()).astype("int")

  n=3

  #ngram3_overlap berechnen
  #Initialisieren der Spalte mit 0
  df["ngram3_overlap"] = 0

  #Loop über die Zeilen des Data Frames:
  for index, row in df.iterrows():

    #Payee & Holder werden von allen Sonderzeichen (.replace('\W', '')) und Leerzeichen (.replace(" ", "")) befreit
    payee = str(row[receiver]).replace('\W', '').replace(" ", "")
    holder = str(row[payer]).replace('\W', '').replace(" ", "")

    #Bildungs des 3grams für Payee und holder
    payee_3gram = [payee[i:i+n] for i in range(len(payee)-n+1)]
    holder_3gram = [holder[i:i+n] for i in range(len(holder)-n+1)]

    #Wir bestimmen die Overlaps zwischen den beiden Sets von 3grams und speichern diese als Liste 
    overlap = list(set(payee_3gram).intersection(set(holder_3gram)))


    #Wir berechnen den Overlap als Anteil der Overlap Menge an der 3Gram Menge des kürzeren der beiden Strings --> diesen Wert setzen wir dann in die entsprechnde Zeile & Spalte im df ein
    df.iloc[index, df.columns.get_loc("ngram3_overlap")] = len(overlap)/min([len(payee_3gram), len(holder_3gram)])

  #Löschung der Hilfsspalten so dass nur die Spalte Eigenueberweisung im ursprünglichen df auftaucht
  del df["is_identical"]
  del df["at_leat_one_NaN"]
  del df["ngram3_overlap"]

  return df



In [14]:
final = Eigenueberweisung(df, receiver="payee_payer_name", payer = "holder_name")